# 🎯 Multi-Agent Writing Studio

- Agent 1 — Professional Writer
- Agent 2 — Creative & Fun Writer
- Agent 3 — Short & Direct Writer
- Agent 4 — Judge Agent (picks the best version)

The system will:

1. Ask each agent to write a short paragraph on the same topic
2. Collect the outputs
3. Have the Judge Agent pick ONE as the best
4. Print the winning version

In [ ]:
import os
from openai import AsyncOpenAI
from agents import set_default_openai_client, Agent, Runner, set_tracing_disabled
from dotenv import load_dotenv
import asyncio

load_dotenv(override=True)

aval_api_key=os.getenv("AVALAI_API_KEY")

In [ ]:
# set up custom client
custom_client = AsyncOpenAI(
    base_url="https://api.avalai.ir/v1",
    api_key=aval_api_key
)

set_default_openai_client(custom_client)
set_tracing_disabled(True)

In [ ]:
# --- Define the agent instructions ---
instructions_professional = "You write in a formal, professional tone. Your explanation is clear and structured."
instructions_creative = "You write in a fun, creative, engaging tone using simple language and friendly energy."
instructions_direct = "You write very short, direct, to-the-point explanations with no extra words."

judge_instructions = """
You are the Judge Agent.
Your job is to select the SINGLE best paragraph from the ones given to you.
Do not write your own paragraph.
Do not explain your choice.
Output ONLY the winning paragraph.
"""

# --- Create the agents ---
professional_writer = Agent(
    name="Professional Writer",
    instructions=instructions_professional,
    model="gpt-4o-mini"
)

creative_writer = Agent(
    name="Creative Writer",
    instructions=instructions_creative,
    model="gpt-4o-mini"
)

direct_writer = Agent(
    name="Direct Writer",
    instructions=instructions_direct,
    model="gpt-4o-mini"
)

judge = Agent(
    name="Judge",
    instructions=judge_instructions,
    model="gpt-4o-mini"
)

In [ ]:
# --- Run the workflow ---
topic = "Explain what AI is for high-school students."

results = await asyncio.gather(
    Runner.run(professional_writer, topic),
    Runner.run(creative_writer, topic),
    Runner.run(direct_writer, topic),
)
outputs = [result.final_output for result in results]

writings = "AI internship emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

result = await Runner.run(judge, writings)

print("🏆 Winning Paragraph:\n")
print(result.final_output)
